# Extracting Data from Companies House Electronic Records

Companies house receives 75% of its records in XBRL or iXBRL format, a glorified tagged xml document that should allow for easy automated extraction of statistics.

The software in this repo was developed after reading of this (American) example:
https://www.codeproject.com/Articles/1227765/Parsing-XBRL-with-Python

The functions for doing so are hosted in the module xbrl_parser.py

Both xbrl_parser.py and this script have a number of python package dependencies so expect to have to install some things.


## Returned dict schema for html/xml sourced data

A practical note:  Apart from explicitly elevated metadata, all extracted values are stored in a list of "elements" within the returned dict.  Each element is itself a dict, containing the name and value of the discovered data along with fields unit and date for metadata.

# Setup (import modules, set up a helper function for getting filepaths)

In [1]:
import xbrl_parser as xp
import os
import numpy as np
import pandas as pd
import importlib

def get_filepaths(directory):

    """ Helper function - 
    Get all of the filenames in a directory that
    end in htm* or xml.
    Under the assumption that all files within
    the folder are financial records. """

    files = [directory + "/" + filename
                for filename in os.listdir(directory)
                    if (("htm" in filename.lower()) or ("xml" in filename.lower())) ]
    return(files)

# Extracting data from documents

We'll import the module, and process some files


In [2]:
# Get all the filenames from the example folder
files = get_filepaths("./example_data_XBRL_iXBRL")

# There's 379 examples currently
files[0:7]

['./example_data_XBRL_iXBRL/Prod224_0042_00958610_20160930.xml',
 './example_data_XBRL_iXBRL/Prod223_2125_09749826_20170831.html',
 './example_data_XBRL_iXBRL/Prod223_2125_09170142_20170831.html',
 './example_data_XBRL_iXBRL/Prod224_0042_03237381_20160831.xml',
 './example_data_XBRL_iXBRL/Prod223_2125_09900460_20161231.html',
 './example_data_XBRL_iXBRL/Prod223_2125_09652609_20180331.html',
 './example_data_XBRL_iXBRL/Prod223_2125_09722743_20170831.html']

In [ ]:
# Reload the xbrl_parser module (don't need this normally, it's just useful for me
# for iterative testing of changes)
importlib.reload(xp)

# try getting the first file (an XML, or XBRL, file)
doc = xp.process_account(files[0])

# display for fun
doc

In [ ]:
# try getting the second file (an HTML, or iXBRL, file)
doc2 = xp.process_account(files[1])

# display for fun
doc2

# Retrieve elements

In [ ]:
# Loop through the document, retrieving any element with a matching name
for element in doc['elements']:
    if element['name'] == 'netassetsliabilitiesincludingpensionassetliability':
        print(element)

In [ ]:
# Extract the all the data to long-thin table format for use with SQL
# Note, tables from docs should be appendable to one another to create
# tables of all data
xp.flatten_data(doc).head(15)

In [ ]:
# Finally, build a table of all variables from all example (digital) documents
# This can take a while

# Empty table awaiting results
results = pd.DataFrame()

# For every file
for file in files:
    
    # Read the file
    doc = xp.process_account(file)
    
    # tabulate the results
    doc_df = xp.flatten_data(doc)
    
    # append to table
    results = results.append(doc_df)

In [ ]:
results.head(15)

That's ~380 files extracted to obtain ~ 22,000 variables - on average 60 variables per record.  As you've just seen though, extraction can take a while!  Searching through the documents using BeautifulSoup can take a long time, especially where chasing element links to get information on units.  Hopefully this is the sort of thing that can be optimised in future, or it'll be rendered irrelevant by Moore's Law.

In [ ]:
results.to_csv("example_extracted_XBRL_data.csv", index=False)

# Get summary variables

These I've implemented to work off the MongoDB/Dict representation of the data that the scraping code returns.  It's assumed that if you wish to work with the "flattened" SQL-compatible data instead you can develop your own queries :)

In [ ]:
index = 3
doc = xp.process_account(files[index])

# This tries to add up every variable it can find in a list of variable names
test = xp.summarise_by_sum(doc, ["fixedassets",
                                 "currentassets",
                                 "intangibleassets",
                                 "tangiblefixedassets",
                                 "intangiblefixedassets",
                                 "investmentsfixedassets",
                                 "cashbankinhand",
                                 "cashbankonhand",
                                 "cashbank",
                                 "cashonhand",
                                 "cashinhand",
                                 "calledupsharecapitalnotpaidnotexpressedascurrentasset",
                                 "otherdebtors"])
test

In [ ]:
# This returns the first variable it finds in a prioritised list
# Here I've gone looking for net assets/liabilities
test = xp.summarise_by_priority(doc, ["netassetsliabilitiesincludingpensionasset",
                                      "netassetsliabilityexcludingpensionasset",
                                      "netassetsliabilities",
                                      "totalassetslesscurrentliabilities",
                                      "netcurrentassetsliabilities"])
test

In [ ]:
# Here I've applied it to shareholder funds/equity
test = xp.summarise_by_priority(doc, ["shareholderfunds",
                                      "equity",
                                      "capitalandreserves"])
test

In [ ]:
# This one just tries to return all named variables
test = xp.summarise_set(doc, ["creditors",
                              "debtors",
                              'accountstypefullorabbreviated',
                              'descriptionprincipalactivities',
                              'accountingstandardsapplied',
                              'entitytradingstatus'])
test

# Finding consolidated status

It turns out that "consolidated" will be tricky - variables exist for it in xbrl/ixbrl docs but they're missleading.  A lot of companies have such a variable and then set the value to "False".

In [17]:
allvars = pd.read_csv("all_variables_sources.csv")

allvars.head()

,Unnamed: 0,Element Count,Element Name,Of Which are Numerical,source
0,12,258094.0,shareholderfunds,258094.0,uk-gaap-full
1,14,240395.0,netassetsliabilitiesincludingpensionassetliabi...,240395.0,uk-gaap-full
2,19,160318.0,creditorsduewithinoneyear,160318.0,uk-gaap-full
3,23,139584.0,sharecapitalallottedcalleduppaid,139584.0,uk-gaap-full
4,24,120765.0,cashbankinhand,120765.0,uk-gaap-full


In [ ]:
consolidation_vars = allvars[allvars['Element Name'].str.contains("consoli")]
consolidation_vars.sort_values("Element Count", ascending=False)

In [ ]:
consolidation_vars = allvars[allvars['Element Name'].str.contains("parent")]
consolidation_vars.sort_values("Element Count", ascending=False)

In [ ]:
consolidation_vars = allvars[allvars['Element Name'].str.contains("accou")]
consolidation_vars.sort_values("Element Count", ascending=False)

In [ ]:
consolidation_vars = allvars[allvars['Element Name'].str.contains("activit")]
consolidation_vars.sort_values("Element Count", ascending=False)

In [ ]:
# manual filter to relevant vars from remaining
for each in consolidation_vars['Element Name']:
    print(each)

In [ ]:
# Built a truth table; for each variable what value should it have to mark a company as consolidated?
consolidation_var_table = {
    "includedinconsolidationsubsidiary":True,
    "investmententityrequiredtoapplyexceptionfromconsolidationtruefalse":True,
    "subsidiaryunconsolidatedtruefalse":False,
    "descriptionreasonwhyentityhasnotpreparedconsolidatedfinancialstatements":"exist",
    "consolidationpolicy":"exist",
    "nameparententity":"exist",
    "amountsowedtoparentundertakingwithinoneyear":"exist",
    "scopeaccounts":"exist"
    }

In [3]:
import pymongo

cl = pymongo.MongoClient()
db = cl['CH_records']
col = db['digital_record_scrapes']

In [4]:
col.find().count()

/home/martin/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


3015767

In [ ]:
docs = col.find().limit(300000)

In [ ]:
results = pd.DataFrame()

for doc in docs:
    
    for each in doc['elements']:
    
        if each['name'] in consolidation_var_table.keys():
        
            res = {"record":doc['doc_companieshouseregisterednumber'],
                   "variable":each['name'],
                   "value":each['value'],
                   "truthval":consolidation_var_table[each['name']]}
        
            results = results.append(res, ignore_index=True)

In [ ]:
results.head()

In [ ]:
results['variable'].unique()

In [ ]:
results[['variable', 'value', 'record']].groupby(['variable', 'value']).agg('count').to_csv("example_consolidation_variables.csv")

In [ ]:
results[['variable', 'value', 'record']].groupby(['variable', 'value']).agg('count')

# Extracting principal activities

In [5]:
docs = col.find(no_cursor_timeout=True)

descriptions = pd.DataFrame()

counter = 0
for doc in docs:
    
    try:
        for each in doc['elements']:
    
            if each['name'] == "descriptionprincipalactivities":
        
                res = {"companieshouseregisterednumber":doc['doc_companieshouseregisterednumber'],
                       "balancesheetdate":doc['doc_balancesheetdate'],
                       "descriptionprincipalactivities":each['value']}
        
                descriptions = descriptions.append(res, ignore_index=True)
                
                counter = counter + 1
                
                if counter % 10000 == 0:
                    print(counter)
                    
                    if counter > 10000:
                        with open("companieshousedescriptions.csv", "a") as f:
                            descriptions.to_csv(f, header=False)
                            print("appended data")

                    else:
                        with open("companieshousedescriptions.csv", "w") as f:
                            descriptions.to_csv(f)
                            print("created_data")
                    
                    descriptions = pd.DataFrame()
                    
                break
    
        
    except:
        pass

10000
created_data
20000
appended data
30000
appended data
40000
appended data
50000
appended data
60000
appended data
70000
appended data
80000
appended data
90000
appended data
100000
appended data
110000
appended data
120000
appended data
130000
appended data
140000
appended data
150000
appended data
160000
appended data
170000
appended data
180000
appended data
190000
appended data
200000
appended data
210000
appended data
220000
appended data
230000
appended data
240000
appended data
250000
appended data
260000
appended data
270000
appended data
280000
appended data
290000
appended data
300000
appended data
310000
appended data
320000
appended data
330000
appended data
340000
appended data
350000
appended data
360000
appended data
370000
appended data
380000
appended data
390000
appended data
400000
appended data
410000
appended data
420000
appended data
430000
appended data
440000
appended data
450000
appended data
460000
appended data
470000
appended data
480000
appended data
49

In [6]:
descriptions = pd.read_csv("companieshousedescriptions.csv")

descriptions.head()

,Unnamed: 0,balancesheetdate,companieshouseregisterednumber,descriptionprincipalactivities
0,0,2017-06-30,08028272,The principal activity of the company is Motor...
1,1,2017-06-30,04761288,No description of principal activity
2,2,2017-08-31,10314494,No description of principal activity
3,3,2016-12-31,03127240,No description of principal activity
4,4,2017-06-30,05735108,No description of principal activity


In [7]:
len(descriptions['descriptionprincipalactivities'].unique())

120931

In [16]:
descriptions[['companieshouseregisterednumber', 'descriptionprincipalactivities']].\
    groupby('descriptionprincipalactivities').agg('count').\
    sort_values('companieshouseregisterednumber', ascending=False).head(20)

,companieshouseregisterednumber
descriptionprincipalactivities,
No description of principal activity,522712
No description of principal activities is disclosed,4940
No principle activity is included,2427
The company is dormant and has not traded during the year.,2067
The principal activity of the company during the year under review was .,1706
The principal activity of the company in the year under review was that of Consultancy.,1280
The principal activity of the company is Information technology consultancy activities,925
The company is dormant and has not traded during the year or subsequent to the year end.,855
IT contracting,851


In [19]:
allvars

,Unnamed: 0,Element Count,Element Name,Of Which are Numerical,source
0,12,258094.0,shareholderfunds,258094.0,uk-gaap-full
1,14,240395.0,netassetsliabilitiesincludingpensionassetliabi...,240395.0,uk-gaap-full
2,19,160318.0,creditorsduewithinoneyear,160318.0,uk-gaap-full
3,23,139584.0,sharecapitalallottedcalleduppaid,139584.0,uk-gaap-full
4,24,120765.0,cashbankinhand,120765.0,uk-gaap-full
5,27,90836.0,calledupsharecapital,90836.0,uk-gaap-full
6,29,87313.0,profitlossaccountreserve,87313.0,uk-gaap-full
7,33,57669.0,tangiblefixedassets,57669.0,uk-gaap-full
8,37,47536.0,creditorsdueafteroneyear,47536.0,uk-gaap-full
9,38,47438.0,tangiblefixedassetscostorvaluation,47438.0,uk-gaap-full
